In [1]:
from gurobipy import *
from prettytable import PrettyTable
from math import pi
import numpy as np

In [2]:
OPF = Model('3 bus OPF')

In [3]:
#Sets
G = 3 # Generators
B = 3 # Bus
L = 3 #Lines
Gen = range(G)
Bus = range(B)
Line = range(L)

### Input the susceptance for each transmission line

In [4]:
B1 = {(0,1):-10, (0,2):-8, (1,2):-5}

### Generate the graph

In [5]:
y = list(B1.keys())
x=list(map (lambda t: (t[1], t[0]), y))
B2=dict(zip(x,B1.values()))
B = {**B1, **B2}

In [6]:
L = {0:200, 1:550, 2:100} # electircal Load (MW)
Sbase = 100 # Power Base (MVA)

In [7]:
GenAt = {Gen[0]:{'a':0.001562 ,'b':7.92 ,'c':561, 'Pmin':150 ,'Pmax':600},
         Gen[1]:{'a':0.001940 ,'b':7.85 ,'c':310, 'Pmin':100 ,'Pmax':400},
         Gen[2]:{'a':0.004820 ,'b':7.97 ,'c':78,  'Pmin':50 , 'Pmax':200}}

In [8]:
Pmin = [(GenAt[i]['Pmin']) for i in Gen]
Pmax = [(GenAt[i]['Pmax']) for i in Gen]
g = [a for a in Gen]
b = [a for a in Bus]

In [9]:
P = OPF.addVars(g, lb = Pmin, ub = Pmax, name = 'Pgen')
Delta = OPF.addVars(b, lb = -6, ub = 6, name = 'Delta')
Pij = OPF.addVars(B,lb = -1000,ub = 1000, name = 'PF')

In [10]:
PowerFlow = OPF.addConstrs((Pij[(i,j)] == Sbase*(Delta[i]-Delta[j])*(B[(i,j)]) for (i,j) in Pij.keys()), name = 'PowerFlow')
NetBus = OPF.addConstrs( (P[i] - (L[i])  == Pij.sum(i,'*') for i in Gen ), name = 'NetBus')

Delta[0].lb = 0
Delta[0].ub = 0

In [11]:
OF = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)

In [12]:
OPF.setObjective(OF, GRB.MINIMIZE)
OPF.Params.Method=-1
OPF.setParam( 'OutputFlag', True )
OPF.optimize()



status = OPF.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    OPF.computeIIS()
    OPF.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 9 rows, 12 columns and 27 nonzeros
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 8e+00]
  QObjective range [3e-03, 1e-02]
  Bounds range     [6e+00, 1e+03]
  RHS range        [1e+02, 6e+02]
Presolve removed 4 rows and 5 columns
Presolve time: 0.01s
Presolved: 5 rows, 7 columns, 15 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.000e+00
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.57973258e+06 -2.17125949e+06  2.50e+03 9.93e+02  1.07e+06     0s
   1   8.20297369e+03 -7.59920431e+05  2.50e-03 9.93e-0

In [13]:
OPF.printStats()


Statistics for model 3 bus OPF :
  Linear constraint matrix    : 9 Constrs, 12 Vars, 27 NZs
  Matrix coefficient range    : [ 1, 1000 ]
  Objective coefficient range : [ 7.85, 7.97 ]
  Variable bound range        : [ 6, 1000 ]
  RHS coefficient range       : [ 100, 550 ]


In [14]:
OPF.getVars()

[<gurobi.Var Pgen[0] (value 393.16983694657904)>,
 <gurobi.Var Pgen[1] (value 334.60375531242556)>,
 <gurobi.Var Pgen[2] (value 122.22640774079191)>,
 <gurobi.Var Delta[0] (value 0.0)>,
 <gurobi.Var Delta[1] (value 0.15817759660199887)>,
 <gurobi.Var Delta[2] (value 0.04374030043086364)>,
 <gurobi.Var PF[0,1] (value 158.17759660199886)>,
 <gurobi.Var PF[0,2] (value 34.99224034469091)>,
 <gurobi.Var PF[1,2] (value -57.21864808561452)>,
 <gurobi.Var PF[1,0] (value -158.17759660199886)>,
 <gurobi.Var PF[2,0] (value -34.99224034469091)>,
 <gurobi.Var PF[2,1] (value 57.21864808551595)>]